# Day 4: Optimal Execution - Almgren-Chriss Model

## Learning Objectives
- Understand the trade-off between market impact and timing risk
- Learn the Almgren-Chriss optimal execution framework
- Implement the model to find optimal trading trajectories
- Analyze how risk aversion affects execution strategies

## Overview

When executing large orders, traders face a fundamental dilemma:
- **Execute quickly**: Minimize exposure to price volatility but incur high market impact costs
- **Execute slowly**: Reduce market impact but face timing risk from price movements

The **Almgren-Chriss model** (2000) provides an elegant framework for finding the optimal balance between these competing objectives.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from dataclasses import dataclass
from typing import Tuple, List
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("Libraries loaded successfully!")

---
## 1. Model Setup and Assumptions

### Problem Statement
A trader needs to liquidate $X$ shares over a time horizon $T$, divided into $N$ discrete trading periods.

### Key Variables
- $X$: Initial position (shares to liquidate)
- $T$: Total time horizon
- $N$: Number of trading periods
- $\tau = T/N$: Length of each period
- $x_k$: Shares remaining after period $k$ (where $x_0 = X$ and $x_N = 0$)
- $n_k = x_{k-1} - x_k$: Shares traded in period $k$

### Price Dynamics
The price evolves according to:
$$S_k = S_{k-1} + \sigma\sqrt{\tau}\xi_k - \tau g\left(\frac{n_k}{\tau}\right)$$

Where:
- $\sigma$: Volatility (annualized)
- $\xi_k \sim N(0,1)$: Random price shocks
- $g(v)$: **Permanent impact** function (affects all future prices)

### Execution Price
The actual execution price includes temporary impact:
$$\tilde{S}_k = S_{k-1} - h\left(\frac{n_k}{\tau}\right)$$

Where $h(v)$ is the **temporary impact** function (only affects current trade).

---
## 2. Linear Impact Model

Almgren-Chriss use **linear impact functions**:

### Permanent Impact
$$g(v) = \gamma v$$
- $\gamma$: Permanent impact parameter
- Represents information leakage or order flow toxicity

### Temporary Impact  
$$h(v) = \epsilon \cdot \text{sign}(v) + \eta v$$
- $\epsilon$: Fixed cost component (bid-ask spread)
- $\eta$: Linear temporary impact parameter

For simplicity, we often ignore the fixed cost and use:
$$h(v) = \eta v$$

In [ ]:
@dataclass
class AlmgrenChrissParams:
    """Parameters for the Almgren-Chriss optimal execution model."""
    
    # Position and time
    X: float          # Initial shares to liquidate
    T: float          # Time horizon (in days)
    N: int            # Number of trading periods
    
    # Market parameters
    S0: float         # Initial stock price
    sigma: float      # Daily volatility
    
    # Impact parameters
    gamma: float      # Permanent impact parameter
    eta: float        # Temporary impact parameter
    epsilon: float    # Fixed cost (bid-ask spread)
    
    # Risk aversion
    lambda_: float    # Risk aversion parameter
    
    @property
    def tau(self) -> float:
        """Time step size."""
        return self.T / self.N
    
    def __repr__(self):
        return (f"AlmgrenChrissParams(X={self.X:,.0f}, T={self.T}, N={self.N}, "
                f"S0=${self.S0:.2f}, σ={self.sigma:.2%}, λ={self.lambda_})")

In [ ]:
# Example parameters - liquidating 100,000 shares over 5 days
params = AlmgrenChrissParams(
    X=100_000,        # 100,000 shares
    T=5.0,            # 5 trading days
    N=20,             # 20 trading periods (4 per day)
    S0=50.0,          # $50 per share
    sigma=0.02,       # 2% daily volatility
    gamma=2.5e-7,     # Permanent impact
    eta=2.5e-6,       # Temporary impact
    epsilon=0.0625,   # Half spread ($0.0625)
    lambda_=1e-6      # Risk aversion
)

print(params)
print(f"\nTime step (τ): {params.tau:.2f} days")
print(f"Total position value: ${params.X * params.S0:,.0f}")

---
## 3. The Optimization Problem

### Implementation Shortfall
The **implementation shortfall** (IS) is the difference between the paper portfolio value and actual execution proceeds:

$$IS = X \cdot S_0 - \sum_{k=1}^{N} n_k \cdot \tilde{S}_k$$

### Expected Shortfall and Variance
Under the linear impact model:

**Expected Shortfall:**
$$E[IS] = \frac{1}{2}\gamma X^2 + \epsilon X + \eta \sum_{k=1}^{N} \frac{n_k^2}{\tau}$$

**Variance of Shortfall:**
$$Var[IS] = \sigma^2 \sum_{k=1}^{N} \tau \cdot x_k^2$$

### Mean-Variance Objective
The trader minimizes:
$$\min_{\{n_k\}} \quad E[IS] + \lambda \cdot Var[IS]$$

Where $\lambda$ is the **risk aversion parameter**.

In [ ]:
def compute_expected_shortfall(n: np.ndarray, params: AlmgrenChrissParams) -> float:
    """
    Compute expected implementation shortfall.
    
    E[IS] = 0.5 * gamma * X^2 + epsilon * X + eta * sum(n_k^2 / tau)
    """
    permanent_impact = 0.5 * params.gamma * params.X**2
    fixed_cost = params.epsilon * params.X
    temporary_impact = params.eta * np.sum(n**2) / params.tau
    
    return permanent_impact + fixed_cost + temporary_impact


def compute_variance_shortfall(n: np.ndarray, params: AlmgrenChrissParams) -> float:
    """
    Compute variance of implementation shortfall.
    
    Var[IS] = sigma^2 * tau * sum(x_k^2)
    where x_k is the remaining position after trade k.
    """
    # Compute remaining positions: x_k = X - cumsum(n)[:k]
    x = params.X - np.cumsum(n)
    x = np.insert(x, 0, params.X)[:-1]  # x_0 to x_{N-1} (positions at start of each period)
    
    return params.sigma**2 * params.tau * np.sum(x**2)


def objective_function(n: np.ndarray, params: AlmgrenChrissParams) -> float:
    """
    Mean-variance objective: E[IS] + lambda * Var[IS]
    """
    expected = compute_expected_shortfall(n, params)
    variance = compute_variance_shortfall(n, params)
    
    return expected + params.lambda_ * variance

---
## 4. Analytical Solution

The Almgren-Chriss model has a **closed-form solution**!

### Optimal Trajectory
The optimal remaining position at time $t_k = k\tau$ is:

$$x_k^* = X \cdot \frac{\sinh(\kappa(T - t_k))}{\sinh(\kappa T)}$$

Where $\kappa$ is the **urgency parameter**:
$$\kappa = \sqrt{\frac{\lambda \sigma^2}{\eta}}$$

### Optimal Trading Rate
$$n_k^* = x_{k-1}^* - x_k^* = X \cdot \frac{2\sinh(\frac{1}{2}\kappa\tau)}{\sinh(\kappa T)} \cdot \cosh\left(\kappa\left(T - t_{k-1/2}\right)\right)$$

Where $t_{k-1/2} = (k - 1/2)\tau$ is the midpoint of period $k$.

In [ ]:
def compute_kappa(params: AlmgrenChrissParams) -> float:
    """
    Compute the urgency parameter kappa.
    
    kappa = sqrt(lambda * sigma^2 / eta)
    
    Higher kappa = more urgent execution (front-loaded)
    Lower kappa = more patient execution (uniform)
    """
    return np.sqrt(params.lambda_ * params.sigma**2 / params.eta)


def optimal_trajectory(params: AlmgrenChrissParams) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Compute the optimal execution trajectory using the closed-form solution.
    
    Returns:
        t: Time points (0, tau, 2*tau, ..., T)
        x: Optimal remaining position at each time
        n: Optimal trade sizes for each period
    """
    kappa = compute_kappa(params)
    tau = params.tau
    T = params.T
    X = params.X
    N = params.N
    
    # Time points
    t = np.linspace(0, T, N + 1)
    
    # Optimal remaining position: x_k = X * sinh(kappa*(T-t_k)) / sinh(kappa*T)
    if kappa * T < 1e-10:  # Risk-neutral case (kappa -> 0)
        x = X * (1 - t / T)
    else:
        x = X * np.sinh(kappa * (T - t)) / np.sinh(kappa * T)
    
    # Ensure boundary conditions
    x[0] = X
    x[-1] = 0
    
    # Optimal trades: n_k = x_{k-1} - x_k
    n = np.diff(-x)  # Negative because x is decreasing
    n = -n  # Make positive (we're selling)
    
    return t, x, n


# Compute optimal trajectory
t, x_opt, n_opt = optimal_trajectory(params)

print(f"Urgency parameter (κ): {compute_kappa(params):.4f}")
print(f"\nOptimal trade sizes (first 5):")
for i in range(5):
    print(f"  Period {i+1}: {n_opt[i]:,.0f} shares")

In [ ]:
def plot_trajectory(t: np.ndarray, x: np.ndarray, n: np.ndarray, 
                    params: AlmgrenChrissParams, title_suffix: str = ""):
    """
    Visualize the optimal execution trajectory.
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Plot 1: Remaining position over time
    ax1 = axes[0]
    ax1.plot(t, x / 1000, 'b-', linewidth=2, marker='o', markersize=4)
    ax1.fill_between(t, 0, x / 1000, alpha=0.3)
    ax1.set_xlabel('Time (days)', fontsize=12)
    ax1.set_ylabel('Remaining Position (000s shares)', fontsize=12)
    ax1.set_title(f'Optimal Execution Trajectory {title_suffix}', fontsize=13)
    ax1.set_xlim(0, params.T)
    ax1.set_ylim(0, params.X / 1000 * 1.05)
    ax1.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
    
    # Add TWAP for comparison
    twap_x = params.X * (1 - t / params.T)
    ax1.plot(t, twap_x / 1000, 'r--', linewidth=1.5, label='TWAP', alpha=0.7)
    ax1.legend()
    
    # Plot 2: Trade sizes per period
    ax2 = axes[1]
    periods = np.arange(1, len(n) + 1)
    colors = plt.cm.Blues(np.linspace(0.4, 0.9, len(n)))
    ax2.bar(periods, n / 1000, color=colors, edgecolor='navy', alpha=0.8)
    ax2.axhline(y=params.X / params.N / 1000, color='red', linestyle='--', 
                linewidth=1.5, label='TWAP rate')
    ax2.set_xlabel('Trading Period', fontsize=12)
    ax2.set_ylabel('Trade Size (000s shares)', fontsize=12)
    ax2.set_title(f'Optimal Trade Sizes by Period {title_suffix}', fontsize=13)
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics
    print(f"\n{'='*50}")
    print(f"Execution Summary {title_suffix}")
    print(f"{'='*50}")
    print(f"Total shares: {params.X:,.0f}")
    print(f"Time horizon: {params.T} days ({params.N} periods)")
    print(f"Average trade size: {np.mean(n):,.0f} shares")
    print(f"First trade: {n[0]:,.0f} shares ({n[0]/params.X*100:.1f}% of total)")
    print(f"Last trade: {n[-1]:,.0f} shares ({n[-1]/params.X*100:.1f}% of total)")
    print(f"Front-loading ratio: {n[0]/n[-1]:.2f}x")


plot_trajectory(t, x_opt, n_opt, params)

---
## 5. Impact of Risk Aversion

The risk aversion parameter $\lambda$ controls the trade-off:

- **$\lambda \to 0$ (Risk-neutral)**: TWAP strategy (uniform execution)
- **$\lambda \to \infty$ (Very risk-averse)**: Immediate execution

Let's visualize how different risk aversion levels affect the optimal strategy.

In [ ]:
def compare_risk_aversions(base_params: AlmgrenChrissParams, 
                          lambdas: List[float]):
    """
    Compare optimal trajectories for different risk aversion levels.
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    colors = plt.cm.viridis(np.linspace(0.1, 0.9, len(lambdas)))
    
    results = []
    
    for lambda_, color in zip(lambdas, colors):
        # Create params with this lambda
        params = AlmgrenChrissParams(
            X=base_params.X, T=base_params.T, N=base_params.N,
            S0=base_params.S0, sigma=base_params.sigma,
            gamma=base_params.gamma, eta=base_params.eta, 
            epsilon=base_params.epsilon, lambda_=lambda_
        )
        
        t, x, n = optimal_trajectory(params)
        kappa = compute_kappa(params)
        
        # Compute costs
        e_is = compute_expected_shortfall(n, params)
        var_is = compute_variance_shortfall(n, params)
        
        results.append({
            'lambda': lambda_,
            'kappa': kappa,
            'E[IS]': e_is,
            'Std[IS]': np.sqrt(var_is),
            'first_trade_pct': n[0] / base_params.X * 100
        })
        
        label = f'λ={lambda_:.0e} (κ={kappa:.3f})'
        axes[0].plot(t, x / 1000, color=color, linewidth=2, label=label)
        axes[1].plot(np.arange(1, len(n)+1), n / 1000, color=color, 
                    linewidth=2, marker='o', markersize=3, label=label)
    
    # Format plots
    axes[0].set_xlabel('Time (days)', fontsize=12)
    axes[0].set_ylabel('Remaining Position (000s)', fontsize=12)
    axes[0].set_title('Trajectory Comparison', fontsize=13)
    axes[0].legend(fontsize=9)
    axes[0].set_xlim(0, base_params.T)
    
    axes[1].set_xlabel('Trading Period', fontsize=12)
    axes[1].set_ylabel('Trade Size (000s)', fontsize=12)
    axes[1].set_title('Trade Size Comparison', fontsize=13)
    axes[1].legend(fontsize=9)
    
    plt.tight_layout()
    plt.show()
    
    # Results table
    df = pd.DataFrame(results)
    df['E[IS]'] = df['E[IS]'].apply(lambda x: f"${x:,.0f}")
    df['Std[IS]'] = df['Std[IS]'].apply(lambda x: f"${x:,.0f}")
    df['first_trade_pct'] = df['first_trade_pct'].apply(lambda x: f"{x:.1f}%")
    
    print("\nComparison of Different Risk Aversion Levels:")
    print(df.to_string(index=False))


# Compare different risk aversion levels
lambdas = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4]
compare_risk_aversions(params, lambdas)

---
## 6. Efficient Frontier of Execution

Different values of $\lambda$ trace out an **efficient frontier** in expected cost vs. risk space.

In [ ]:
def plot_efficient_frontier(base_params: AlmgrenChrissParams, 
                           n_points: int = 50):
    """
    Plot the efficient frontier of execution strategies.
    """
    lambdas = np.logspace(-10, -3, n_points)
    
    expected_costs = []
    std_costs = []
    kappas = []
    
    for lambda_ in lambdas:
        params = AlmgrenChrissParams(
            X=base_params.X, T=base_params.T, N=base_params.N,
            S0=base_params.S0, sigma=base_params.sigma,
            gamma=base_params.gamma, eta=base_params.eta, 
            epsilon=base_params.epsilon, lambda_=lambda_
        )
        
        _, _, n = optimal_trajectory(params)
        
        e_is = compute_expected_shortfall(n, params)
        var_is = compute_variance_shortfall(n, params)
        
        expected_costs.append(e_is)
        std_costs.append(np.sqrt(var_is))
        kappas.append(compute_kappa(params))
    
    # Convert to basis points of notional
    notional = base_params.X * base_params.S0
    expected_bps = np.array(expected_costs) / notional * 10000
    std_bps = np.array(std_costs) / notional * 10000
    
    # Plot
    fig, ax = plt.subplots(figsize=(10, 7))
    
    scatter = ax.scatter(std_bps, expected_bps, c=np.log10(lambdas), 
                        cmap='viridis', s=50, alpha=0.8)
    ax.plot(std_bps, expected_bps, 'k-', alpha=0.3, linewidth=1)
    
    # Add colorbar
    cbar = plt.colorbar(scatter, ax=ax)
    cbar.set_label('log₁₀(λ)', fontsize=11)
    
    # Mark special points
    # TWAP (approximately lambda -> 0)
    ax.annotate('TWAP\n(risk-neutral)', xy=(std_bps[-1], expected_bps[-1]),
                xytext=(std_bps[-1]+5, expected_bps[-1]-2),
                fontsize=10, ha='left',
                arrowprops=dict(arrowstyle='->', color='gray'))
    
    # Immediate execution (approximately lambda -> infinity)
    ax.annotate('Aggressive\n(risk-averse)', xy=(std_bps[0], expected_bps[0]),
                xytext=(std_bps[0]-10, expected_bps[0]+3),
                fontsize=10, ha='right',
                arrowprops=dict(arrowstyle='->', color='gray'))
    
    ax.set_xlabel('Execution Risk (Std Dev in bps)', fontsize=12)
    ax.set_ylabel('Expected Cost (bps)', fontsize=12)
    ax.set_title('Efficient Frontier of Execution Strategies', fontsize=14)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nNotional value: ${notional:,.0f}")
    print(f"Expected cost range: {min(expected_bps):.1f} to {max(expected_bps):.1f} bps")
    print(f"Risk range: {min(std_bps):.1f} to {max(std_bps):.1f} bps")


plot_efficient_frontier(params)

---
## 7. Monte Carlo Simulation

Let's simulate actual execution to verify our analytical results and see the distribution of outcomes.

In [ ]:
def simulate_execution(n: np.ndarray, params: AlmgrenChrissParams, 
                       n_simulations: int = 10000, seed: int = 42) -> dict:
    """
    Monte Carlo simulation of execution with price uncertainty.
    
    Returns dict with simulation results.
    """
    np.random.seed(seed)
    
    N = params.N
    tau = params.tau
    sigma = params.sigma
    gamma = params.gamma
    eta = params.eta
    S0 = params.S0
    X = params.X
    
    # Paper portfolio value (benchmark)
    paper_value = X * S0
    
    # Storage for simulation results
    implementation_shortfalls = []
    final_prices = []
    total_proceeds = []
    
    for sim in range(n_simulations):
        # Generate random price shocks
        xi = np.random.standard_normal(N)
        
        # Simulate price path and execution
        S = S0  # Current market price
        proceeds = 0
        
        for k in range(N):
            # Trading rate
            v_k = n[k] / tau
            
            # Execution price (before trade, with temporary impact)
            S_exec = S - eta * v_k
            
            # Proceeds from this trade
            proceeds += n[k] * S_exec
            
            # Price evolves (permanent impact + random)
            S = S + sigma * np.sqrt(tau) * xi[k] - gamma * v_k * tau
        
        # Implementation shortfall
        IS = paper_value - proceeds
        
        implementation_shortfalls.append(IS)
        final_prices.append(S)
        total_proceeds.append(proceeds)
    
    IS_array = np.array(implementation_shortfalls)
    
    return {
        'IS': IS_array,
        'E[IS]': np.mean(IS_array),
        'Std[IS]': np.std(IS_array),
        'IS_bps': IS_array / paper_value * 10000,
        'proceeds': np.array(total_proceeds),
        'final_prices': np.array(final_prices)
    }


# Run simulation for optimal strategy
sim_results = simulate_execution(n_opt, params, n_simulations=10000)

print("Monte Carlo Simulation Results (10,000 simulations)")
print("="*50)
print(f"Expected IS: ${sim_results['E[IS]']:,.0f} ({np.mean(sim_results['IS_bps']):.2f} bps)")
print(f"Std Dev IS: ${sim_results['Std[IS]']:,.0f} ({np.std(sim_results['IS_bps']):.2f} bps)")
print(f"\nAnalytical E[IS]: ${compute_expected_shortfall(n_opt, params):,.0f}")
print(f"Analytical Std[IS]: ${np.sqrt(compute_variance_shortfall(n_opt, params)):,.0f}")

In [ ]:
def plot_simulation_results(sim_results: dict, params: AlmgrenChrissParams):
    """
    Visualize Monte Carlo simulation results.
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    IS_bps = sim_results['IS_bps']
    
    # Histogram of implementation shortfall
    ax1 = axes[0]
    ax1.hist(IS_bps, bins=50, density=True, alpha=0.7, color='steelblue', edgecolor='white')
    ax1.axvline(np.mean(IS_bps), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(IS_bps):.1f} bps')
    ax1.axvline(np.percentile(IS_bps, 5), color='orange', linestyle=':', linewidth=2, label=f'5th pctl: {np.percentile(IS_bps, 5):.1f} bps')
    ax1.axvline(np.percentile(IS_bps, 95), color='orange', linestyle=':', linewidth=2, label=f'95th pctl: {np.percentile(IS_bps, 95):.1f} bps')
    ax1.set_xlabel('Implementation Shortfall (bps)', fontsize=12)
    ax1.set_ylabel('Density', fontsize=12)
    ax1.set_title('Distribution of Implementation Shortfall', fontsize=13)
    ax1.legend(fontsize=10)
    
    # Box plot comparison with TWAP
    ax2 = axes[1]
    
    # Simulate TWAP
    twap_n = np.ones(params.N) * params.X / params.N
    twap_results = simulate_execution(twap_n, params, n_simulations=10000, seed=42)
    
    data_to_plot = [IS_bps, twap_results['IS_bps']]
    bp = ax2.boxplot(data_to_plot, labels=['Optimal', 'TWAP'], patch_artist=True)
    
    colors = ['steelblue', 'coral']
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)
    
    ax2.set_ylabel('Implementation Shortfall (bps)', fontsize=12)
    ax2.set_title('Optimal vs TWAP Comparison', fontsize=13)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Summary comparison
    print("\nStrategy Comparison:")
    print(f"{'Metric':<20} {'Optimal':>15} {'TWAP':>15}")
    print("-" * 50)
    print(f"{'Mean IS (bps)':<20} {np.mean(IS_bps):>15.2f} {np.mean(twap_results['IS_bps']):>15.2f}")
    print(f"{'Std Dev (bps)':<20} {np.std(IS_bps):>15.2f} {np.std(twap_results['IS_bps']):>15.2f}")
    print(f"{'5th Percentile':<20} {np.percentile(IS_bps, 5):>15.2f} {np.percentile(twap_results['IS_bps'], 5):>15.2f}")
    print(f"{'95th Percentile':<20} {np.percentile(IS_bps, 95):>15.2f} {np.percentile(twap_results['IS_bps'], 95):>15.2f}")


plot_simulation_results(sim_results, params)

---
## 8. Parameter Sensitivity Analysis

Let's examine how the optimal strategy changes with different market conditions.

In [ ]:
def sensitivity_analysis(base_params: AlmgrenChrissParams):
    """
    Analyze sensitivity of optimal execution to key parameters.
    """
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Sensitivity to volatility
    ax = axes[0, 0]
    sigmas = np.linspace(0.005, 0.05, 20)
    first_trade_pcts = []
    
    for sigma in sigmas:
        p = AlmgrenChrissParams(
            X=base_params.X, T=base_params.T, N=base_params.N,
            S0=base_params.S0, sigma=sigma,
            gamma=base_params.gamma, eta=base_params.eta,
            epsilon=base_params.epsilon, lambda_=base_params.lambda_
        )
        _, _, n = optimal_trajectory(p)
        first_trade_pcts.append(n[0] / p.X * 100)
    
    ax.plot(sigmas * 100, first_trade_pcts, 'b-', linewidth=2, marker='o', markersize=4)
    ax.set_xlabel('Daily Volatility (%)', fontsize=11)
    ax.set_ylabel('First Trade (% of total)', fontsize=11)
    ax.set_title('Sensitivity to Volatility', fontsize=12)
    ax.grid(True, alpha=0.3)
    ax.axvline(base_params.sigma * 100, color='red', linestyle='--', alpha=0.7, label='Base case')
    ax.legend()
    
    # 2. Sensitivity to temporary impact (eta)
    ax = axes[0, 1]
    etas = np.logspace(-7, -5, 20)
    first_trade_pcts = []
    
    for eta in etas:
        p = AlmgrenChrissParams(
            X=base_params.X, T=base_params.T, N=base_params.N,
            S0=base_params.S0, sigma=base_params.sigma,
            gamma=base_params.gamma, eta=eta,
            epsilon=base_params.epsilon, lambda_=base_params.lambda_
        )
        _, _, n = optimal_trajectory(p)
        first_trade_pcts.append(n[0] / p.X * 100)
    
    ax.semilogx(etas, first_trade_pcts, 'g-', linewidth=2, marker='s', markersize=4)
    ax.set_xlabel('Temporary Impact (η)', fontsize=11)
    ax.set_ylabel('First Trade (% of total)', fontsize=11)
    ax.set_title('Sensitivity to Temporary Impact', fontsize=12)
    ax.grid(True, alpha=0.3)
    ax.axvline(base_params.eta, color='red', linestyle='--', alpha=0.7, label='Base case')
    ax.legend()
    
    # 3. Sensitivity to time horizon
    ax = axes[1, 0]
    horizons = np.linspace(1, 10, 20)
    kappas = []
    expected_costs = []
    
    for T in horizons:
        p = AlmgrenChrissParams(
            X=base_params.X, T=T, N=int(T * 4),  # Keep 4 periods per day
            S0=base_params.S0, sigma=base_params.sigma,
            gamma=base_params.gamma, eta=base_params.eta,
            epsilon=base_params.epsilon, lambda_=base_params.lambda_
        )
        _, _, n = optimal_trajectory(p)
        kappas.append(compute_kappa(p))
        expected_costs.append(compute_expected_shortfall(n, p))
    
    ax.plot(horizons, np.array(expected_costs) / (base_params.X * base_params.S0) * 10000, 
            'purple', linewidth=2, marker='^', markersize=4)
    ax.set_xlabel('Time Horizon (days)', fontsize=11)
    ax.set_ylabel('Expected Cost (bps)', fontsize=11)
    ax.set_title('Sensitivity to Time Horizon', fontsize=12)
    ax.grid(True, alpha=0.3)
    ax.axvline(base_params.T, color='red', linestyle='--', alpha=0.7, label='Base case')
    ax.legend()
    
    # 4. Cost breakdown by urgency
    ax = axes[1, 1]
    lambdas = np.logspace(-9, -4, 30)
    temp_impacts = []
    timing_risks = []
    
    for lambda_ in lambdas:
        p = AlmgrenChrissParams(
            X=base_params.X, T=base_params.T, N=base_params.N,
            S0=base_params.S0, sigma=base_params.sigma,
            gamma=base_params.gamma, eta=base_params.eta,
            epsilon=base_params.epsilon, lambda_=lambda_
        )
        _, _, n = optimal_trajectory(p)
        
        # Temporary impact cost component
        temp_impact = p.eta * np.sum(n**2) / p.tau
        temp_impacts.append(temp_impact)
        
        # Timing risk
        var_is = compute_variance_shortfall(n, p)
        timing_risks.append(np.sqrt(var_is))
    
    notional = base_params.X * base_params.S0
    ax.semilogx(lambdas, np.array(temp_impacts) / notional * 10000, 
                'b-', linewidth=2, label='Temporary Impact Cost')
    ax.semilogx(lambdas, np.array(timing_risks) / notional * 10000, 
                'r-', linewidth=2, label='Timing Risk (Std Dev)')
    ax.set_xlabel('Risk Aversion (λ)', fontsize=11)
    ax.set_ylabel('Cost Component (bps)', fontsize=11)
    ax.set_title('Cost Components vs Risk Aversion', fontsize=12)
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


sensitivity_analysis(params)

---
## 9. Complete Execution Framework Class

Let's package everything into a reusable class.

In [ ]:
class AlmgrenChrissExecutor:
    """
    Complete implementation of the Almgren-Chriss optimal execution model.
    """
    
    def __init__(self, params: AlmgrenChrissParams):
        self.params = params
        self._compute_optimal_strategy()
    
    def _compute_optimal_strategy(self):
        """Compute the optimal execution strategy."""
        self.kappa = compute_kappa(self.params)
        self.time_points, self.trajectory, self.trade_sizes = optimal_trajectory(self.params)
        self.expected_cost = compute_expected_shortfall(self.trade_sizes, self.params)
        self.variance = compute_variance_shortfall(self.trade_sizes, self.params)
        self.std_dev = np.sqrt(self.variance)
    
    @property
    def notional(self) -> float:
        """Total notional value of the trade."""
        return self.params.X * self.params.S0
    
    @property
    def expected_cost_bps(self) -> float:
        """Expected cost in basis points."""
        return self.expected_cost / self.notional * 10000
    
    @property
    def std_dev_bps(self) -> float:
        """Standard deviation in basis points."""
        return self.std_dev / self.notional * 10000
    
    def get_schedule(self) -> pd.DataFrame:
        """Return the execution schedule as a DataFrame."""
        schedule = pd.DataFrame({
            'Period': range(1, self.params.N + 1),
            'Time_Start': self.time_points[:-1],
            'Time_End': self.time_points[1:],
            'Shares_to_Trade': self.trade_sizes,
            'Cumulative_Shares': np.cumsum(self.trade_sizes),
            'Remaining_After': self.trajectory[1:],
            'Pct_of_Total': self.trade_sizes / self.params.X * 100
        })
        return schedule
    
    def simulate(self, n_simulations: int = 10000, seed: int = None) -> dict:
        """Run Monte Carlo simulation."""
        return simulate_execution(self.trade_sizes, self.params, n_simulations, seed)
    
    def summary(self):
        """Print a summary of the optimal strategy."""
        print("="*60)
        print("ALMGREN-CHRISS OPTIMAL EXECUTION SUMMARY")
        print("="*60)
        print(f"\nTrade Details:")
        print(f"  Shares to liquidate: {self.params.X:,.0f}")
        print(f"  Initial price: ${self.params.S0:.2f}")
        print(f"  Notional value: ${self.notional:,.0f}")
        print(f"  Time horizon: {self.params.T} days ({self.params.N} periods)")
        
        print(f"\nMarket Parameters:")
        print(f"  Daily volatility: {self.params.sigma:.2%}")
        print(f"  Permanent impact (γ): {self.params.gamma:.2e}")
        print(f"  Temporary impact (η): {self.params.eta:.2e}")
        
        print(f"\nOptimal Strategy:")
        print(f"  Risk aversion (λ): {self.params.lambda_:.2e}")
        print(f"  Urgency (κ): {self.kappa:.4f}")
        print(f"  First trade: {self.trade_sizes[0]:,.0f} shares ({self.trade_sizes[0]/self.params.X*100:.1f}%)")
        print(f"  Last trade: {self.trade_sizes[-1]:,.0f} shares ({self.trade_sizes[-1]/self.params.X*100:.1f}%)")
        
        print(f"\nExpected Costs:")
        print(f"  Expected shortfall: ${self.expected_cost:,.0f} ({self.expected_cost_bps:.2f} bps)")
        print(f"  Risk (std dev): ${self.std_dev:,.0f} ({self.std_dev_bps:.2f} bps)")
        print("="*60)
    
    def plot(self):
        """Visualize the optimal strategy."""
        plot_trajectory(self.time_points, self.trajectory, 
                       self.trade_sizes, self.params)


# Example usage
executor = AlmgrenChrissExecutor(params)
executor.summary()

In [ ]:
# Display the execution schedule
schedule = executor.get_schedule()
print("\nExecution Schedule:")
print(schedule.to_string(index=False))

In [ ]:
# Plot the strategy
executor.plot()

---
## 10. Practical Considerations and Extensions

### Limitations of the Basic Model
1. **Linear impact assumption**: Real market impact is often non-linear
2. **Constant parameters**: Volatility and liquidity vary throughout the day
3. **No information**: Doesn't account for alpha signals
4. **Continuous trading**: Assumes continuous trading ability

### Common Extensions
1. **Non-linear impact models**: Power-law or square-root impact
2. **Time-varying parameters**: Intraday liquidity patterns
3. **With alpha**: Trading off impact vs. signal decay
4. **Multi-asset**: Coordinated execution of portfolio trades
5. **Stochastic volatility**: Adapting to changing market conditions

In [ ]:
# Extension: Non-linear (square root) temporary impact
def optimal_trajectory_sqrt_impact(params: AlmgrenChrissParams, 
                                   impact_exponent: float = 0.5) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Numerical optimization for non-linear temporary impact.
    
    h(v) = eta * sign(v) * |v|^alpha
    
    Uses scipy optimization since no closed-form solution exists.
    """
    N = params.N
    X = params.X
    tau = params.tau
    
    def objective(n):
        # Expected cost with power-law impact
        permanent = 0.5 * params.gamma * X**2
        temporary = params.eta * np.sum(np.abs(n / tau)**impact_exponent * np.abs(n))
        expected = permanent + temporary
        
        # Variance (same as linear model)
        x = X - np.cumsum(n)
        x = np.insert(x, 0, X)[:-1]
        variance = params.sigma**2 * tau * np.sum(x**2)
        
        return expected + params.lambda_ * variance
    
    # Constraints: sum of trades = X, all trades positive
    constraints = {'type': 'eq', 'fun': lambda n: np.sum(n) - X}
    bounds = [(0, X) for _ in range(N)]
    
    # Initial guess: TWAP
    n0 = np.ones(N) * X / N
    
    result = minimize(objective, n0, method='SLSQP', 
                     constraints=constraints, bounds=bounds)
    
    n_opt = result.x
    t = np.linspace(0, params.T, N + 1)
    x = X - np.cumsum(np.insert(n_opt, 0, 0))
    
    return t, x, n_opt


# Compare linear vs square-root impact
t_linear, x_linear, n_linear = optimal_trajectory(params)
t_sqrt, x_sqrt, n_sqrt = optimal_trajectory_sqrt_impact(params, impact_exponent=0.5)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(t_linear, x_linear / 1000, 'b-', linewidth=2, label='Linear Impact', marker='o', markersize=3)
axes[0].plot(t_sqrt, x_sqrt / 1000, 'r--', linewidth=2, label='Square-Root Impact', marker='s', markersize=3)
axes[0].set_xlabel('Time (days)', fontsize=12)
axes[0].set_ylabel('Remaining Position (000s)', fontsize=12)
axes[0].set_title('Trajectory: Linear vs Square-Root Impact', fontsize=13)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

periods = np.arange(1, len(n_linear) + 1)
width = 0.35
axes[1].bar(periods - width/2, n_linear / 1000, width, label='Linear Impact', color='steelblue', alpha=0.8)
axes[1].bar(periods + width/2, n_sqrt / 1000, width, label='Square-Root Impact', color='coral', alpha=0.8)
axes[1].set_xlabel('Trading Period', fontsize=12)
axes[1].set_ylabel('Trade Size (000s)', fontsize=12)
axes[1].set_title('Trade Sizes: Linear vs Square-Root Impact', fontsize=13)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nWith square-root impact, optimal strategy is more front-loaded")
print("because the marginal cost of trading decreases with size.")

---
## 11. Key Takeaways

### The Almgren-Chriss Model
1. **Balances two competing objectives**: Minimizing market impact (favors slow trading) vs. minimizing timing risk (favors fast trading)

2. **Key insight**: The optimal strategy depends on the ratio of volatility-adjusted risk aversion to temporary impact ($\kappa = \sqrt{\lambda\sigma^2/\eta}$)

3. **Closed-form solution**: Optimal trajectory follows a hyperbolic sine function

4. **Risk aversion determines strategy shape**:
   - Low $\lambda$: TWAP-like uniform execution
   - High $\lambda$: Front-loaded aggressive execution

5. **Efficient frontier**: Different risk-return trade-offs available by varying $\lambda$

### Practical Applications
- **Benchmark for execution algorithms**: Compare actual execution to AC optimal
- **Parameter estimation**: Calibrate impact parameters from trade data
- **Strategy design**: Use as foundation for more complex models
- **Risk management**: Understand execution risk exposure

---
## 12. Exercises

1. **Parameter Calibration**: Given a set of historical trades, estimate the permanent and temporary impact parameters.

2. **Intraday Patterns**: Modify the model to account for U-shaped intraday liquidity patterns.

3. **With Alpha**: Extend the model to include an expected price drift (alpha signal) that decays over time.

4. **Multi-Asset**: Implement coordinated execution for a portfolio of correlated assets.

5. **Adaptive Execution**: Create an algorithm that updates the execution schedule based on realized market conditions.

In [ ]:
# Exercise starter: Execution with alpha signal
def optimal_trajectory_with_alpha(params: AlmgrenChrissParams, 
                                  alpha: float,      # Expected daily return (positive = price going up)
                                  alpha_decay: float # Rate of alpha decay
                                  ) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Optimal execution with alpha signal.
    
    When selling with positive alpha (price expected to rise), 
    we should trade more slowly to capture the appreciation.
    
    When selling with negative alpha (price expected to fall),
    we should trade more aggressively to avoid losses.
    """
    # YOUR IMPLEMENTATION HERE
    # Hint: Add alpha contribution to the objective function
    # Expected proceeds increase if we wait when alpha > 0
    pass


print("Exercise: Implement optimal execution with alpha signal")
print("Consider how the presence of alpha should change the urgency of execution.")

---
## References

1. Almgren, R., & Chriss, N. (2000). *Optimal execution of portfolio transactions*. Journal of Risk, 3, 5-40.

2. Almgren, R. (2003). *Optimal execution with nonlinear impact functions and trading-enhanced risk*. Applied Mathematical Finance, 10(1), 1-18.

3. Gatheral, J. (2010). *No-dynamic-arbitrage and market impact*. Quantitative Finance, 10(7), 749-759.

4. Cartea, Á., Jaimungal, S., & Penalva, J. (2015). *Algorithmic and High-Frequency Trading*. Cambridge University Press.